1. Description
group_map(), group_modify() and group_walk() are purrr-style functions that can be used to iterate on grouped tibbles.

2. Usage

group_map ( .tbl,  .f,  . . . , keep = FALSE)

group_modify ( .tbl,  .f,  . . . , keep = FALSE)

group_walk( .tbl,  .f,  . . .  )

3. Arguments
.tbl
A grouped tibble

.f
A function or formula to apply to each group. It must return a data frame.

If a function,  it is used as is. It should have at least 2 formal arguments.

If a formula, e.g.  ~ head(.x), it is converted to a function.

In the formular, you can use
         . or .x to refer to the subset of rows of .tbl for the given group
         .y to refer to the key, a one row tibble with one column per grouping variable that identifies the group
         
. . . 
Additional arguments passed on to .f

keep
are the grouping variables kept in .x

4. Details
Use group_modify() when summarize() is too limited, in terms of what you need to do and return for each group. 
group_modify() is good for " data frame in, data frame out". If that is too limited, you need to use a nested or split workflow.
group_modify() is an evolution of do().

Each conceptual group of the data frame is exposed to the function .f with two pieces of information:
            (1) The subset of the data for the group, exposed as .x
            (2) The key, a tibble with exactly one row and columns for each grouping variable, exposed as .y
For completeness, group_modify(), group_map and group_walk() also work on ungrouped data frame, in that case the function is applied to the entire data frame ( exposed as .x ), and .y is a one row tibble with no column, consistently with group_keys().

5. Values

group_modify() returns a grouped tibble. In that case  .f  must return a data frame. 
group_map () returns a list of results from calling .f on each group
group_walk() calls .f for side effects and returns the input  .tbl ,  invisibly





In [5]:
library(dplyr)
library(tibble)
data(mtcars)

rownames_to_column(mtcars) -> mtcars
mtcars

rowname,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2


In [4]:
# return a list
mtcars %>% group_by(cyl)  %>% group_map( ~ head(.x  ,  2L ) )

rowname,mpg,disp,hp,drat,wt,qsec,vs,am,gear,carb
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Datsun 710,22.8,108.0,93,3.85,2.32,18.61,1,1,4,1
Merc 240D,24.4,146.7,62,3.69,3.19,20.00,1,0,4,2
rowname,mpg,disp,hp,drat,wt,qsec,vs,am,gear,carb
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21,160,110,3.9,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21,160,110,3.9,2.875,17.02,0,1,4,4
rowname,mpg,disp,hp,drat,wt,qsec,vs,am,gear,carb
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Hornet Sportabout,18.7,360,175,3.15,3.44,17.02,0,0,3,2


In [10]:
# return a tibble grouped by 'cyl' with 2 rows per group
# the grouping data is recalculated

mtcars %>% group_by( cyl ) %>% group_modify ( ~ head(.x , 2L))

data(iris)
# a list of tibbles
iris %>% group_by( Species ) %>% group_map( ~ broom :: tidy(lm( Petal.Length ~ Sepal.Length, data = .x )))

# a restructed grouped tibble
iris %>% group_by( Species ) %>% group_modify( ~ broom :: tidy(lm( Petal.Length ~ Sepal.Length, data = .x )))


cyl,rowname,mpg,disp,hp,drat,wt,qsec,vs,am,gear,carb
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,Datsun 710,22.8,108.0,93,3.85,2.320,18.61,1,1,4,1
4,Merc 240D,24.4,146.7,62,3.69,3.190,20.00,1,0,4,2
6,Mazda RX4,21.0,160.0,110,3.90,2.620,16.46,0,1,4,4
6,Mazda RX4 Wag,21.0,160.0,110,3.90,2.875,17.02,0,1,4,4
8,Hornet Sportabout,18.7,360.0,175,3.15,3.440,17.02,0,0,3,2
8,Duster 360,14.3,360.0,245,3.21,3.570,15.84,0,0,3,4


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.8030518,0.3438781,2.335281,0.02375647
Sepal.Length,0.1316317,0.0685269,1.920876,0.06069778
term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.1851155,0.51421351,0.3599974,7.204283e-01
Sepal.Length,0.6864698,0.08630708,7.9538056,2.586190e-10
term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.6104680,0.41710685,1.463577,1.498279e-01


Species,term,estimate,std.error,statistic,p.value
<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
setosa,(Intercept),0.8030518,0.34387807,2.3352806,2.375647e-02
setosa,Sepal.Length,0.1316317,0.06852690,1.9208760,6.069778e-02
versicolor,(Intercept),0.1851155,0.51421351,0.3599974,7.204283e-01
versicolor,Sepal.Length,0.6864698,0.08630708,7.9538056,2.586190e-10
virginica,(Intercept),0.6104680,0.41710685,1.4635770,1.498279e-01
virginica,Sepal.Length,0.7500808,0.06302606,11.9011203,6.297786e-16


In [11]:
# a list of vectors

iris %>% group_by( Species ) %>% group_map(~ quantile(.x$Petal.Length, probs = c( 0.25, 0.5, 0.75  )) )

[[1]]
  25%   50%   75% 
1.400 1.500 1.575 

[[2]]
 25%  50%  75% 
4.00 4.35 4.60 

[[3]]
  25%   50%   75% 
5.100 5.550 5.875

In [12]:
# to use group_modify() the lambda must return a data frame
iris %>% group_by(Species) %>% 
              group_modify( ~ {
                  quantile( .x$Petal.Length, probs = c(0.25, 0.5, 0.75  )) %>% 
                  tibble::enframe( name = "prob", value = "quantile")
              })


Species,prob,quantile
<fct>,<chr>,<dbl>
setosa,25%,1.400
setosa,50%,1.500
setosa,75%,1.575
versicolor,25%,4.000
versicolor,50%,4.350
versicolor,75%,4.600
virginica,25%,5.100
virginica,50%,5.550
virginica,75%,5.875


In [13]:
iris %>% group_by(Species ) %>% 
              group_modify( ~ {
                  .x %>% purrr::map_dfc(fivenum ) %>% 
                  mutate(nms = c( "min", "Q1", "median", "Q3", "max"))
              })


Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,nms
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
setosa,4.3,2.3,1.00,0.1,min
setosa,4.8,3.2,1.40,0.2,Q1
setosa,5.0,3.4,1.50,0.2,median
setosa,5.2,3.7,1.60,0.3,Q3
setosa,5.8,4.4,1.90,0.6,max
versicolor,4.9,2.0,3.00,1.0,min
versicolor,5.6,2.5,4.00,1.2,Q1
versicolor,5.9,2.8,4.35,1.3,median
versicolor,6.3,3.0,4.60,1.5,Q3


In [15]:
#group_walk() is for side effects
dir.create( temp <- tempfile())

iris %>% group_by(Species) %>% 
              group_walk(~ write.csv(.x,  file = file.path(temp,  paste0( .y$Species,   ".csv"))))

list.files ( temp, pattern = "csv$")

unlink(temp, recursive = TRUE)



Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


[1] "setosa.csv"     "versicolor.csv" "virginica.csv"

In [14]:
#group_modify() and ungrouped data frames

mtcars %>% group_modify( ~ head(.x,  2L))

rowname,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21,6,160,110,3.9,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4
